In [1]:
from pyspark.sql.types import *
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

In [2]:
spark = SparkSession.builder.appName("Transform").getOrCreate()


25/08/09 16:27:04 WARN Utils: Your hostname, Acer resolves to a loopback address: 127.0.1.1; using 192.168.1.6 instead (on interface wlp4s0)
25/08/09 16:27:04 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/08/09 16:27:05 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:

rSubmissionBronzeSchema = StructType([
    StructField("id", StringType(), True),
    StructField("title", StringType(), True),
    StructField("selftext", StringType(), True),
    StructField("url", StringType(), True),
    StructField("permalink", StringType(), True),
    StructField("domain", StringType(), True),
    StructField("post_hint", StringType(), True),
    StructField("author", StringType(), True),
    StructField("author_fullname", StringType(), True),
    StructField("created_utc", LongType(), True),
    StructField("subreddit", StringType(), True),
    StructField("subreddit_id", StringType(), True),
    StructField("subreddit_name_prefixed", StringType(), True),
    StructField("subreddit_type", StringType(), True),
    StructField("subreddit_subscribers", IntegerType(), True),
    StructField("score", IntegerType(), True),
    StructField("num_comments", IntegerType(), True),
    StructField("total_awards_received", IntegerType(), True),
    StructField("edited", BooleanType(), True),
    StructField("locked", BooleanType(), True),
    StructField("spoiler", BooleanType(), True),
    StructField("over_18", BooleanType(), True),
    StructField("stickied", BooleanType(), True),   
    StructField("retrieved_on", LongType(), True),
    StructField("is_original_content", BooleanType(), True),
    StructField("link_flair_text", StringType(), True)
])


In [4]:
rCommentBronzeSchema = StructType([
    StructField("id", StringType(), True),
    StructField("body", StringType(), True),
    StructField("created_utc", LongType(), True),
    StructField("edited", BooleanType(), True),
    StructField("score", IntegerType(), True),
    StructField("author", StringType(), True),
    StructField("author_fullname", StringType(), True),
    StructField("author_created_utc", LongType(), True),
    StructField("parent_id", StringType(), True),
    StructField("link_id", StringType(), True),
    StructField("is_submitter", BooleanType(), True),
    StructField("permalink", StringType(), True),
    StructField("subreddit", StringType(), True),
    StructField("subreddit_id", StringType(), True),
    StructField("subreddit_name_prefixed", StringType(), True),
    StructField("subreddit_type", StringType(), True),
    StructField("total_awards_received", IntegerType(), True),
    StructField("controversiality", IntegerType(), True),
    StructField("retrieved_on", LongType(), True),
    StructField("stickied", BooleanType(), True)
])


In [5]:
# df_sub = spark.read.format("json").schema(rSubmissionBronzeSchema)\
# .load("submission.jsonl")
df_sub = spark.read.format("json").schema(rSubmissionBronzeSchema)\
.load("../data/RS_reddit.jsonl")



In [6]:
df_cmt = spark.read.format("json").schema(rCommentBronzeSchema)\
.load("../data/RC_reddit.jsonl")


In [7]:
df_sub.show(2)

25/08/09 16:27:33 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+-------+--------------------+---------+--------------------+--------------------+-------------------+---------+---------+---------------+-----------+----------+------------+-----------------------+--------------+---------------------+-----+------------+---------------------+------+------+-------+-------+--------+------------+-------------------+---------------+
|     id|               title| selftext|                 url|           permalink|             domain|post_hint|   author|author_fullname|created_utc| subreddit|subreddit_id|subreddit_name_prefixed|subreddit_type|subreddit_subscribers|score|num_comments|total_awards_received|edited|locked|spoiler|over_18|stickied|retrieved_on|is_original_content|link_flair_text|
+-------+--------------------+---------+--------------------+--------------------+-------------------+---------+---------+---------------+-----------+----------+------------+-----------------------+--------------+---------------------+-----+------------+----------------

In [8]:
df_cmt.show(2)

+-------+--------------------+-----------+------+-----+---------+---------------+------------------+----------+----------+------------+--------------------+----------+------------+-----------------------+--------------+---------------------+----------------+------------+--------+
|     id|                body|created_utc|edited|score|   author|author_fullname|author_created_utc| parent_id|   link_id|is_submitter|           permalink| subreddit|subreddit_id|subreddit_name_prefixed|subreddit_type|total_awards_received|controversiality|retrieved_on|stickied|
+-------+--------------------+-----------+------+-----+---------+---------------+------------------+----------+----------+------------+--------------------+----------+------------+-----------------------+--------------+---------------------+----------------+------------+--------+
|mvc2u2o|From the article\...| 1748737579| false|    0| Gari_305|    t2_65fa26pr|              NULL|t3_1l0buai|t3_1l0buai|        true|/r/Futurology/com...|F

In [9]:
def shape(df):
    return (len(df.columns), (df.count()))

In [ ]:
shape(df_sub)

In [342]:
shape(df_cmt)

(20, 6)

In [343]:
from pyspark.sql import functions as F

df_sub = df_sub.withColumn(
    "created_date",
    F.from_unixtime(F.col("created_utc")).cast("timestamp")
)


In [344]:
df_sub.select('created_date').show(5)

+-------------------+
|       created_date|
+-------------------+
|2025-06-01 07:07:25|
|2025-06-01 07:25:33|
|2025-06-01 07:27:18|
|2025-06-01 07:27:41|
|2025-06-01 07:30:13|
+-------------------+
only showing top 5 rows



In [345]:
from pyspark.sql import functions as F

df_cmt = df_cmt.withColumn(
    "created_date",
    F.from_unixtime(F.col("created_utc")).cast("timestamp")
)


In [346]:
df_cmt.select('created_date').show(5)



+-------------------+
|       created_date|
+-------------------+
|2025-06-06 21:29:04|
|2025-06-06 22:16:19|
|2025-06-06 22:13:52|
|2025-06-06 22:31:06|
|2025-06-06 22:31:07|
+-------------------+
only showing top 5 rows



In [347]:
df_accDeleted = df_sub.selectExpr(
    "author", 
    "author_fullname", 
    "author = '[deleted]' AS flag1",
    "author_fullname IS NULL AS flag2",
)
df_accDeleted.show()

+--------------------+---------------+-----+-----+
|              author|author_fullname|flag1|flag2|
+--------------------+---------------+-----+-----+
|           [deleted]|           NULL| true| true|
|            Gari_305|    t2_65fa26pr|false|false|
|           katxwoods|    t2_6h6wd5oz|false|false|
|       Businessmarck|    t2_8mw6j6fe|false|false|
|     Present_Fill412|   t2_p5klht0fl|false|false|
|     Present_Fill412|   t2_p5klht0fl|false|false|
|     Present_Fill412|   t2_p5klht0fl|false|false|
|     Present_Fill412|   t2_p5klht0fl|false|false|
|    IlustriousCoffee|  t2_1khgdtu3js|false|false|
|            Gari_305|    t2_65fa26pr|false|false|
| Careless_Package555|    t2_vorn1pyv|false|false|
|            Gari_305|    t2_65fa26pr|false|false|
|      indig0sixalpha|       t2_ht9gq|false|false|
|  Informal-Rock-2681|  t2_14i3wvhp29|false|false|
|    Clearblueskymind|    t2_abqdnc4d|false|false|
|     Ok_Positive4828|  t2_10jrr0662p|false|false|
|              d-kuro|    t2_4p

In [348]:
df_accDeleted.selectExpr("flag1 AS isTrue") \
             .filter("isTrue") \
             .count()

116

In [349]:
df_accDeleted.selectExpr("flag2 AS isTrue") \
             .filter("isTrue") \
             .count()

116

In [350]:
df_sub = df_sub.selectExpr(
                '*',
                "author='[deleted]' AND author_fullname IS NULL AS acc_deleted"
)

In [351]:
df_sub.select('acc_deleted').show(5)

+-----------+
|acc_deleted|
+-----------+
|       true|
|      false|
|      false|
|      false|
|      false|
+-----------+
only showing top 5 rows



In [352]:
df_sub.filter('acc_deleted').count()

116

In [353]:
shape(df_sub)

(28, 7306)

In [354]:
df_body_removed = df_sub.selectExpr(
        'acc_deleted',
        "selftext='[removed]' AS post_deleted")

In [355]:
df_body_removed.show(2)

+-----------+------------+
|acc_deleted|post_deleted|
+-----------+------------+
|       true|        true|
|      false|       false|
+-----------+------------+
only showing top 2 rows



In [356]:
df_body_removed.filter("post_deleted = true").count()

1149

In [357]:
df_body_removed.filter("post_deleted = true").count() * 100 / df_body_removed.filter("post_deleted = false").count()

18.661685885983434

In [358]:
df_body_removed.filter("acc_deleted = true").count()

116

In [359]:
df_sub = df_sub.selectExpr("*", "selftext = '[removed]' AS post_deleted")
df_sub.show(2)

+-------+--------------------+---------+--------------------+--------------------+-------------------+---------+---------+---------------+-----------+----------+------------+-----------------------+--------------+---------------------+-----+------------+---------------------+------+------+-------+-------+--------+------------+-------------------+---------------+-------------------+-----------+------------+
|     id|               title| selftext|                 url|           permalink|             domain|post_hint|   author|author_fullname|created_utc| subreddit|subreddit_id|subreddit_name_prefixed|subreddit_type|subreddit_subscribers|score|num_comments|total_awards_received|edited|locked|spoiler|over_18|stickied|retrieved_on|is_original_content|link_flair_text|       created_date|acc_deleted|post_deleted|
+-------+--------------------+---------+--------------------+--------------------+-------------------+---------+---------+---------------+-----------+----------+------------+------

In [360]:
df_sub.createOrReplaceTempView("submissions")

df_sub = spark.sql("""
SELECT *,
  (
    SIZE(
      FILTER(
        SPLIT(lower(title), ' '), 
        word -> instr(lower(url), word) > 0
      )
    ) * 1.0 / SIZE(SPLIT(lower(title), ' '))
  ) AS match_ratio,
  (
    SIZE(
      FILTER(
        SPLIT(lower(title), ' '), 
        word -> instr(lower(url), word) > 0
      )
    ) * 1.0 / SIZE(SPLIT(lower(title), ' '))
  ) >= 0.6
  AND domain <> concat('self.', subreddit) AS spam_title_in_url
FROM submissions
""")


In [362]:
df_sub.select("match_ratio", "spam_title_in_url").show()


+--------------+-----------------+
|   match_ratio|spam_title_in_url|
+--------------+-----------------+
|0.000000000000|            false|
|0.571428571429|            false|
|0.941176470588|             true|
|1.000000000000|            false|
|0.222222222222|            false|
|0.500000000000|            false|
|0.500000000000|            false|
|0.000000000000|            false|
|0.700000000000|             true|
|0.550000000000|            false|
|0.583333333333|            false|
|0.363636363636|            false|
|1.000000000000|             true|
|0.545454545455|            false|
|0.466666666667|            false|
|0.034482758621|            false|
|0.090909090909|            false|
|0.000000000000|            false|
|1.000000000000|            false|
|0.750000000000|             true|
+--------------+-----------------+
only showing top 20 rows



In [363]:
shape(df_sub)

(31, 7306)